# TABLE DES MATIERES
* [INFORMATIONS ATTENDUES](#INFORMATIONS-ATTENDUES)
* [SOURCES](#SOURCES)
    * [ANNUAIRE](#ANNUAIRE)
    * [BDD TOPO](#BDD-TOPO)
    * [POTENTIEL GISEMENT SOLAIRE BRUT AU BATI](#POTENTIEL-GISEMENT-SOLAIRE-BRUT-AU-BATI)
* [EVALUATION](#EVALUATION)
    * [COMPLETUDE](#COMPLETUDE)
    * [PROXIMITE](#PROXIMITE)
        * [Similarité cosinus entre deux textes](#Similarité-cosinus-entre-deux-textes) 

Références: 
- https://outline.services.dataforgood.fr/doc/contraintes-a-respecter-pour-linstallation-de-panneaux-solaires-W2W2IaNwKn
- https://data-iau-idf.opendata.arcgis.com/datasets/iau-idf::le-potentiel-solaire-des-toitures/about
- https://outline.services.dataforgood.fr/doc/ressources-algorithme-9afGJyytC8
- https://geocatalogue.apur.org/catalogue/srv/fre/catalog.search#/metadata/urn:apur:potentiel_gisement_solaire_brut_au_bati

# INFORMATIONS ATTENDUES

## Contraintes à respecter
- Logistique
    - Pente du toit
    - Orientation au soleil
    - Latitude
    - Indice de rayonnement solaire local
    - Bâtiments classés
    - Bâtiments en zone non classée
    - Ombre (Bâtiments adjacents & végétations)
    - Eléments présents sur le toit
    - Structure porteuse (est-ce que le bâtiment peut supporter le poids?)
- Réseaux

# Potentiel Solaire - Attributs importants
- surf_util : Surface présentant une irradiation suffisante (> 900 kWh/m².an) et sans obstacles (comme des cheminées, velux, aérations, etc.). Cette surface est réellement utilisable pour l'installation de panneaux solaires.
- indic2 : Valeur numérique correspondant au gisement solaire. Les valeurs possibles sont :
1 : Non favorable
1 : Faible
2 : Intermédiaire
3 : Important
- gisement : Classification du gisement solaire en termes qualitatifs : "important", "intermédiaire", "faible", ou "Non favorable".
- moyenne2 : Potentiel solaire moyen au m² du bâtiment, exprimé en kWh/m².an. Cet indicateur représente la quantité d'énergie solaire reçue sur la surface utile.
- production : La production annuelle théorique en kWh/an, qui est une estimation de la quantité d'énergie solaire que la toiture pourrait produire sur une année.
- forme : Forme de la toiture estimée à partir d'un modèle numérique de surface. La forme de la toiture peut influencer l'ensoleillement et donc le potentiel solaire.
- eq_pano : La surface utile en équivalent panneaux solaires. Cet attribut indique le nombre de panneaux solaires susceptibles d'être installés sur la toiture.
- eq_surf : L'équivalent en superficie de l'équivalent panneau solaire, qui donne une idée de la surface disponible pour l'installation de panneaux solaires.

# Qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO
"un des points importants à regarder en l'état c'est la qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO. Pour le moment c'est une règle sur la proximité géographique qui est utilisée mais il y a d'autres métadonnées qui peuvent nous éclairer je pense. Par exemple :
le champs toponyme du layer zone d'activité de la BD TOPO (ex : "Ecole Elementaire Victor Hugo") avec le champs nom_etablissement de l'annuaire (ex: "Ecole élémentaire Victor Hugo")
le champs nature du layer zone d'activité de la BD TOPO (9 valeurs possibles) avec type_etablissement de l'annuaire (8 valeurs possibles)
En creusant, c'est clairement pas l'égalité à chercher mais peut etre calculer la proximité des valeurs (ou juste les comparer) pour voir si on s'est pas complètement loupé. Ou trouver une piste pour les établissements de l'annuaire sans surface d'activité.
Sachant que j'ai pas regardé tous les champs et il y a peut être d'autres informations à utiliser."

# SOURCES

## ANNUAIRE

In [1]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona
import pandas as pd

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

In [2]:
!extract-sample-data

2025-02-21 08:57:43,118 - INFO - root - C:\Users\eteiw\Desktop\amel\algorithme\potentiel_solaire\extract_sample_data.py - main - GPKG file potentiel-solaire.gpkg already exists, skipping conversion.
2025-02-21 08:57:43,118 - INFO - root - C:\Users\eteiw\Desktop\amel\algorithme\potentiel_solaire\extract_sample_data.py - main - GPKG file potentiel-gisement-solaire-brut-au-bati.gpkg already exists, skipping conversion.
2025-02-21 08:57:43,118 - INFO - root - C:\Users\eteiw\Desktop\amel\algorithme\potentiel_solaire\extract_sample_data.py - main - Folder for BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15.7z already exists, skipping download and extraction.
2025-02-21 08:57:43,118 - INFO - root - C:\Users\eteiw\Desktop\amel\algorithme\potentiel_solaire\extract_sample_data.py - main - Folder for PARCELLAIRE-EXPRESS_1-1__SHP_LAMB93_D093_2024-10-01.7z already exists, skipping download and extraction.
2025-02-21 08:57:43,118 - INFO - root - C:\Users\eteiw\Desktop\amel\algorithme\potentiel_sol

In [3]:
annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
#annuaire.shape
annuaire[annuaire.code_commune=='93066'].sample(15) # Saint-Denis

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,...,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
57662,0930342S,Ecole élémentaire Antoine de Saint-Exupéry,Ecole,Public,3 passage des Ecoles,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932058G,None,None,None,None,POINT (2.37429 48.93674)
65469,0932470E,Ecole maternelle La Lison,Ecole,Public,17 rue du Bailly,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.35591 48.9127)
65442,0931521Y,Ecole maternelle La Saussaie,Ecole,Public,31 allée des Saules,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932609F,None,None,None,None,POINT (2.38131 48.94505)
37503,0932546M,Centre Médico-Scolaire de Saint-Denis,Service Administratif,None,2 rue Guy Môquet,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,0939999N,ANNEXE GEOGRAPHIQUE,None,None,None,None,None,POINT (2.36214 48.94165)
10924,0930393X,Ecole élémentaire Paul Langevin,Ecole,Public,2 rue Guy Môquet,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932609F,None,None,None,None,POINT (2.36214 48.94165)
11149,0932665S,Ecole maternelle Le Corduan,Ecole,Public,12 mail des Maraichers,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.36462 48.91399)
36503,0932613K,Ecole élémentaire Pina Bausch,Ecole,Public,55 rue Charles Michel,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,0932613K,None,None,0931036W,None,None,None,None,POINT (2.34294 48.93274)
11117,0932500M,Ecole élémentaire Maria Casarès,Ecole,Public,3 rue Cristino Garcia,LA PLAINE SAINT DENIS,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.36599 48.91645)
65309,0930306C,Ecole élémentaire Daniel Sorano,Ecole,Public,3 boulevard Félix Faure,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932609F,None,None,None,None,POINT (2.35805 48.93877)
11159,0932738W,Ecole maternelle Rouillon,Ecole,Public,36 avenue Lénine,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932609F,None,None,None,None,POINT (2.36598 48.94175)


## BDD TOPO

In [4]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")

# on filtre sur saint-denis
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)


In [5]:
# Exploration des layers
layers = fiona.listlayers(GPKG)
layers

['troncon_de_route',
 'route_numerotee_ou_nommee',
 'itineraire_autre',
 'troncon_de_voie_ferree',
 'equipement_de_transport',
 'piste_d_aerodrome',
 'aerodrome',
 'point_de_repere',
 'non_communication',
 'point_du_reseau',
 'voie_ferree_nommee',
 'toponymie_transport',
 'batiment',
 'cimetiere',
 'construction_lineaire',
 'construction_ponctuelle',
 'construction_surfacique',
 'reservoir',
 'ligne_orographique',
 'pylone',
 'terrain_de_sport',
 'toponymie_bati',
 'cours_d_eau',
 'troncon_hydrographique',
 'bassin_versant_topographique',
 'plan_d_eau',
 'surface_hydrographique',
 'noeud_hydrographique',
 'detail_hydrographique',
 'toponymie_hydrographie',
 'zone_d_habitation',
 'lieu_dit_non_habite',
 'detail_orographique',
 'toponymie_lieux_nommes',
 'canalisation',
 'ligne_electrique',
 'poste_de_transformation',
 'erp',
 'zone_d_activite_ou_d_interet',
 'toponymie_services_et_activites',
 'voie_nommee',
 'parc_ou_reserve',
 'foret_publique',
 'toponymie_zones_reglementees',
 'haie'

In [6]:
zone_d_activite=gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet")

In [7]:
zone_d_activite[zone_d_activite.categorie=='Science et enseignement']

,cleabs,categorie,nature,nature_detaillee,toponyme,statut_du_toponyme,importance,fictif,etat_de_l_objet,date_creation,date_modification,date_d_apparition,date_de_confirmation,sources,identifiants_sources,methode_d_acquisition_planimetrique,precision_planimetrique,identifiant_voie_ban,nom_commercial,geometry
27,SURFACTI0000000295936983,Science et enseignement,Structure d'accueil pour personnes handicapées,Etablissement et service d'aide par le travail,ESAT Ateliers de Chennevières,Collecté,5,True,En service,2012-02-17 10:09:31.224,2024-08-26 16:16:13.197,2017/01/03,2022-09-29,FINESS 2022,FINESS:940800170,Calculé,20.0,None,None,"MULTIPOLYGON (((666368.3 6855629.2, 666363.3 6..."
33,SURFACTI0000002206611076,Science et enseignement,Autre établissement d'enseignement,Centre de formation continue,Of-Cfa Afec,Collecté,6,True,En service,2020-04-22 17:54:14.640,2022-12-05 12:26:55.418,None,2022-10-19,MEN 2022,MEN:0756046P,Levé non GPS,5.0,None,None,"MULTIPOLYGON (((653850.8 6863186, 653845.8 686..."
36,SURFACTI0000000206971078,Science et enseignement,Lycée,None,Lycée Pour Handicapés votre École chez Vous,Collecté,6,True,En service,2008-10-03 16:13:30.756,2024-07-04 15:12:12.409,None,2024-07-04,MEN 2021,MEN:0754089M,Calculé,20.0,None,None,"MULTIPOLYGON (((654856.5 6862558.5, 654851.5 6..."
38,SURFACTI0000000356611011,Science et enseignement,Enseignement supérieur,None,Atelier Blanche Salant & Paul Weaver,Collecté,6,True,En service,2016-02-18 13:36:30.302,2024-07-09 10:20:43.092,None,2024-07-09,None,None,Calculé,20.0,None,None,"MULTIPOLYGON (((654664 6862999.9, 654659 68629..."
46,SURFACTI0000000106099761,Science et enseignement,Enseignement primaire,Ecole maternelle,École Ganénou Joëlle Msika-Annexe,Collecté,6,True,En service,2007-11-19 09:12:17.280,2024-07-09 17:28:15.581,None,2024-07-09,None,None,Calculé,20.0,None,None,"MULTIPOLYGON (((655351.5 6861455.2, 655346.5 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13339,SURFACTI0000000295401379,Science et enseignement,Enseignement supérieur,None,Institut de Formation en Soins Infirmiers Léon...,Collecté,5,False,En service,2012-02-13 13:27:16.754,2024-12-04 11:48:56.481,None,2024-04-09,MEN 2024,MEN:0951575W/MEN:0951987U,Orthophotographie,3.0,None,None,"MULTIPOLYGON (((655446.4 6876009.5, 655456.6 6..."
13340,SURFACTI0000000003753622,Science et enseignement,Enseignement primaire,Ecole maternelle,École Maternelle Lelong,Collecté,5,False,En service,2006-05-22 16:59:15.638,2024-12-04 11:48:56.481,None,2024-04-09,MEN 2024,MEN:0950613A,BDTopo,2.5,None,None,"MULTIPOLYGON (((654687.9 6877207.5, 654696.3 6..."
13341,SURFACTI0000000003753630,Science et enseignement,Collège,None,Collège Voltaire,Collecté,5,False,En service,2006-05-22 16:59:15.638,2024-12-04 11:48:56.481,None,2024-04-09,MEN 2024,MEN:0951196J,Orthophotographie,3.0,None,None,"MULTIPOLYGON (((654062.5 6877624.8, 654059 687..."
13343,SURFACTI0000002480826612,Science et enseignement,Enseignement primaire,Ecole maternelle,Annexe École Maternelle Marie Pape-Carpantier,Collecté,6,False,En service,2024-12-04 12:46:39.170,NaT,None,2024-12-04,None,MEN:0951263G,Orthophotographie,3.0,None,None,"MULTIPOLYGON (((655281.3 6878630.5, 655280.7 6..."


## POTENTIEL GISEMENT SOLAIRE BRUT AU BATI


Les attributs importants pour le projet sont en **gras** 

- N_SQ_EB: Identifiant informatique séquentiel
- **C_CAINSEE: Code INSEE de la commune/arrondissement**
- **C_ENS_MOY: Ensoleillement annuel moyen (KWh/m²/an)**
- M2_E_IN700: Surface d'ensoleillement inférieure à 700 KWh/m²/an
- M2_E_70_80: Surface d'ensoleillement comprise entre 700 et 800 KWh/m²/an
- M2_E_80_90: Surface d'ensoleillement comprise entre 800 et 900 KWh/m²/an
- **M2_E_90_10: Surface d'ensoleillement comprise entre 900 et 1000 KWh/m²/an**
- **M2_E_S1000: Surface d'ensoleillement supérieure à 1000 KWh/m²/an**
- **M2_E_TOT: Surface totale d'ensoleillement (m²)**
- SHAPE_LENGTH: Périmètre de l’emprise (m)
- SHAPE_AREA: Surface de l’emprise (m²)

In [8]:
potentiel_gisement_solaire = gpd.read_file(DATA_FOLDER / 'potentiel-gisement-solaire-brut-au-bati.geojson')
potentiel_gisement_solaire = potentiel_gisement_solaire.to_crs(4326)

In [9]:
potentiel_gisement_solaire[potentiel_gisement_solaire.c_cainsee==93066]

,geo_point_2d,geo_shape,objectid,n_sq_eb,n_sq_eb_or,c_cainsee,m2_e_in700,m2_e_70_80,m2_e_80_90,m2_e_90_10,m2_e_s1000,m2_e_tot,c_ens_moy,shape_leng,shape_area,geometry
88,"{ ""lon"": 2.3722225145083691, ""lat"": 48.9404461...",None,941255,930299587.0,930299587.0,93066.0,38,22,23,24,31,138,831.0,52.416020,137.239893,"POLYGON ((2.37221 48.94039, 2.37214 48.94038, ..."
283,"{ ""lon"": 2.3610456606653232, ""lat"": 48.9283787...",None,941832,930386417.0,930386417.0,93066.0,24,4,1,0,0,29,466.0,28.430968,28.457771,"POLYGON ((2.36103 48.92841, 2.36107 48.9284, 2..."
859,"{ ""lon"": 2.375205160916229, ""lat"": 48.93637781...",None,943584,930304850.0,930304850.0,93066.0,3,1,0,0,0,4,672.0,10.022848,5.530794,"POLYGON ((2.37523 48.93638, 2.37522 48.93636, ..."
894,"{ ""lon"": 2.3778331638397905, ""lat"": 48.9430036...",None,943683,930298629.0,930298629.0,93066.0,7,1,1,0,0,9,472.0,12.016152,9.018584,"POLYGON ((2.37786 48.943, 2.37783 48.94298, 2...."
895,"{ ""lon"": 2.3787945342325707, ""lat"": 48.9361676...",None,943686,930305161.0,930305161.0,93066.0,4,0,0,0,0,4,234.0,8.552651,4.166027,"POLYGON ((2.37881 48.93616, 2.37879 48.93615, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092902,"{ ""lon"": 2.37915734042992, ""lat"": 48.937699688...",None,1029247,930304932.0,930304932.0,93066.0,7,0,0,0,0,7,271.0,11.673293,8.086631,"POLYGON ((2.37918 48.93769, 2.37913 48.93769, ..."
1092927,"{ ""lon"": 2.3777861095320287, ""lat"": 48.9373595...",None,1029322,930304349.0,930304349.0,93066.0,10,8,12,15,36,81,919.0,48.266526,80.604890,"POLYGON ((2.37786 48.93739, 2.37783 48.93736, ..."
1092928,"{ ""lon"": 2.3592067500392417, ""lat"": 48.9152216...",None,1029326,930309628.0,930309628.0,93066.0,51,0,0,0,0,51,422.0,28.672137,50.266614,"POLYGON ((2.35926 48.91525, 2.35926 48.91519, ..."
1093006,"{ ""lon"": 2.3750083312923853, ""lat"": 48.9441876...",None,1029577,930300021.0,930300021.0,93066.0,0,0,0,1,0,1,852.0,5.691323,1.824023,"POLYGON ((2.37502 48.94418, 2.375 48.94418, 2...."


## POTENTIEL SOLAIRE

In [10]:
potentiel = gpd.read_file(DATA_FOLDER / 'potentiel-solaire.geojson')

In [11]:
potentiel

,objectid,id,nature,surf_util,indic2,gisement,eq_pano,eq_surf,systeme,protection,mos2017,insee,moyenne2,forme,production,mos17,st_area(shape),st_length(shape),geometry
0,1,BATIMENT0000000356480536,Bâtiment industriel,382.50,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,45,95078,1210.328054,plat,42184.047797,5,609.870,108.056166,"POLYGON ((2.03865 49.08767, 2.03867 49.08767, ..."
1,2,BATIMENT0000000356480545,Bâtiment industriel,83.25,2,intermédiaire,10 à 50 panneaux,entre 20 et 115 m2,thermique ou photovoltaïque,0,45,95078,1197.261665,Npans,12199.856914,5,159.180,51.796212,"POLYGON ((2.04011 49.08265, 2.04006 49.08257, ..."
2,3,BATIMENT0000000356475551,Bâtiment industriel,213.75,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,1,45,95211,1204.673158,plat,23463.298631,5,713.220,115.314199,"POLYGON ((2.1148 49.06608, 2.11477 49.06605, 2..."
3,4,BATIMENT0000000317228796,Bâtiment industriel,240.75,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,1,45,95211,1243.157793,toit2pentes,36633.125213,5,729.620,119.633673,"POLYGON ((2.11964 49.06853, 2.11947 49.06864, ..."
4,5,BATIMENT0000000003786796,Bâtiment industriel,132.75,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,1,45,95370,1210.220040,plat,14639.039443,5,305.425,74.696097,"POLYGON ((1.97282 49.14438, 1.97277 49.14436, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,994,BATIMENT0000000003957857,Serre,126.00,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,10,95018,1305.472070,plat,14988.281492,1,737.580,117.842566,"POLYGON ((2.2141 48.95716, 2.21353 48.95719, 2..."
996,995,BATIMENT0000000003957856,Serre,139.50,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,10,95018,1179.596782,plat,14994.137799,1,374.490,98.708164,"POLYGON ((2.21411 48.95717, 2.21412 48.95725, ..."
997,996,BATIMENT0000000003957858,Serre,85.50,2,intermédiaire,10 à 50 panneaux,entre 20 et 115 m2,thermique ou photovoltaïque,0,10,95018,1236.205659,toit2pentes,12937.139463,1,322.970,89.279465,"POLYGON ((2.21413 48.95729, 2.21414 48.95737, ..."
998,997,BATIMENT0000000003953824,Serre,326.25,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,10,95018,1198.183775,plat,35619.487445,1,959.600,144.546231,"POLYGON ((2.21968 48.96245, 2.21988 48.96238, ..."


# EVALUATION

## COMPLETUDE

### Valeurs manquantes

In [12]:
taux_noms_etablissements_manquants=annuaire.nom_etablissement.isnull().sum()/len(annuaire.nom_etablissement)
print(f"Taux de noms d'établissement manquants: {taux_noms_etablissements_manquants} %")

Taux de noms d'établissement manquants: 0.0 %


In [13]:
taux_toponymes_manquants=zone_d_activite.toponyme.isnull().sum()/len(zone_d_activite.toponyme)
print(f"Taux de toponymes*100 manquants: {taux_toponymes_manquants*100} %") 

Taux de toponymes*100 manquants: 8.667965238237938 %


### Couverture temporelle

In [14]:
annuaire[(annuaire.type_etablissement=='Ecole') & (annuaire.code_commune=='93066')].date_maj_ligne.unique()

<DatetimeArray>
['2025-02-03 00:00:00']
Length: 1, dtype: datetime64[ms]

In [15]:
zone_d_activite[zone_d_activite.categorie=='Science et enseignement'][["date_modification"]].sort_values(by='date_modification',ascending=False)

,date_modification
13344,2024-12-05 08:50:36.735
9400,2024-12-04 12:46:39.170
13341,2024-12-04 11:48:56.481
13339,2024-12-04 11:48:56.481
13340,2024-12-04 11:48:56.481
...,...
8539,NaT
8540,NaT
8623,NaT
8788,NaT


## PROXIMITE

Pour voir si les informations sont bien correlées, essayons de calculer la proximité des valeurs (ou juste les comparer) entre sources de données 

In [16]:
# recherche avec suppression préalable des accents (normalisation + encodage + desencodage + minuscule) 
s_annuaire_ecole_1=annuaire[annuaire.nom_etablissement.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()=="ecole elementaire daniel sorano"]
s_annuaire_ecole_1

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,...,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
65309,0930306C,Ecole élémentaire Daniel Sorano,Ecole,Public,3 boulevard Félix Faure,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932609F,None,None,None,None,POINT (2.35805 48.93877)


In [17]:
s_bddtopo_ecole_1=zone_d_activite[zone_d_activite.toponyme.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.contains("ecole elementaire daniel sorano")==True]
s_bddtopo_ecole_1

,cleabs,categorie,nature,nature_detaillee,toponyme,statut_du_toponyme,importance,fictif,etat_de_l_objet,date_creation,date_modification,date_d_apparition,date_de_confirmation,sources,identifiants_sources,methode_d_acquisition_planimetrique,precision_planimetrique,identifiant_voie_ban,nom_commercial,geometry
9968,SURFACTI0000000002555607,Science et enseignement,Enseignement primaire,Groupe scolaire,École Élémentaire Daniel Sorano,Collecté,5,False,En service,2006-05-22 16:31:28.175,2024-04-17 14:57:25.903,None,2024-04-09,MEN 2024,MEN:0930306C,BDTopo,2.5,None,None,"MULTIPOLYGON (((653017.7 6871189.3, 653017.4 6..."


In [18]:
s_annuaire_ecole_1[['nom_etablissement','type_etablissement']]

,nom_etablissement,type_etablissement
65309,Ecole élémentaire Daniel Sorano,Ecole


In [19]:
s_bddtopo_ecole_1[['toponyme','nature']]

,toponyme,nature
9968,École Élémentaire Daniel Sorano,Enseignement primaire


### Similarité cosinus entre deux textes

Nous calculons la similarité cosinus entre deux attributs texte pour déterminer le taux de similarité  
Plus la valeur de la similarité cosinus est :  
- proche de 1, plus les textes sont similaires.  
- proche de 0, plus les textes sont différents.

In [20]:
import nltk
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eteiw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [21]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from unidecode import unidecode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eteiw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import nltk
import nltk


In [23]:
nltk.download('punkt')
nltk.download('stopwords') 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eteiw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eteiw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
import spacy
from unidecode import unidecode

In [25]:
def similarite_cosinus(X,Y):
    # Programme pour mesurer la similarité entre
    # deux phrases en utilisant la similarité cosinus (similarité=1 -> identique, similarité=0 -> pas du tout similaire)

    # Suppression des accents et passage en minuscules
    X = unidecode(X).lower()
    Y = unidecode(Y).lower()
    
    # Tokenisation
    X_list = word_tokenize(X)
    Y_list = word_tokenize(Y)
    
    # sw contient la liste des mots vides
    sw = set(stopwords.words('french'))
    l1 = []
    l2 = []
    
    # Supprimer les mots vides de la chaîne
    X_set = {w for w in X_list if w not in sw}
    Y_set = {w for w in Y_list if w not in sw}
    
    # Former un ensemble contenant les mots-clés des deux chaînes
    rvector = X_set.union(Y_set)
    for w in rvector:
        if w in X_set:
            l1.append(1)  # créer un premier vecteur
        else:
            l1.append(0)
        if w in Y_set:
            l2.append(1) # créer un deuxieme vecteur
        else:
            l2.append(0)
    c = 0
    
    # Formule du cosinus
    for i in range(len(rvector)):
        c += l1[i] * l2[i]
    cosine = c / float((sum(l1) * sum(l2)) ** 0.5)
    return cosine

### **⚠️ Vérification des Données (Attention : Code Modifié) ci-dessous

In [26]:
from unidecode import unidecode

# Liste simplifiée de mots vides
french_stopwords = set(["de", "la", "le", "et", "les", "des", "un", "une", "à", "d'", "l'", "en", "du"])

def similarite_cosinus(X, Y):
    """
    Calculer la similarité cosinus entre deux chaînes.
    """
    # Prétraitement : suppression des accents et passage en minuscules
    X = unidecode(X).lower()
    Y = unidecode(Y).lower()

    # Tokenisation simplifiée (sans dépendances NLTK)
    X_list = [word for word in X.split() if word not in french_stopwords]
    Y_list = [word for word in Y.split() if word not in french_stopwords]

    # Créer un ensemble de mots unique
    rvector = set(X_list).union(set(Y_list))

    # Créer les vecteurs binaires
    l1 = [1 if w in X_list else 0 for w in rvector]
    l2 = [1 if w in Y_list else 0 for w in rvector]

    # Calcul du produit scalaire et des normes
    produit_scalaire = sum(a * b for a, b in zip(l1, l2))
    norme_X = sum(l1) ** 0.5
    norme_Y = sum(l2) ** 0.5

    # Éviter la division par zéro
    if norme_X == 0 or norme_Y == 0:
        return 0.0

    # Calcul de la similarité cosinus
    return produit_scalaire / (norme_X * norme_Y)

# utilisation
X = "Ecole"
Y = "Enseignement primaire"

print(f"similarité ({X} | {Y}): {similarite_cosinus(X, Y)}")


similarité (Ecole | Enseignement primaire): 0.0


In [27]:
print(f"X: {X}, type: {type(X)}")
print(f"Y: {Y}, type: {type(Y)}")


X: Ecole, type: <class 'str'>
Y: Enseignement primaire, type: <class 'str'>


In [28]:

X=s_annuaire_ecole_1['nom_etablissement'].values[0]
Y=s_bddtopo_ecole_1['toponyme'].values[0]

print(f"similarité ({X}|{Y}): {similarite_cosinus(X,Y)}")

similarité (Ecole élémentaire Daniel Sorano|École Élémentaire Daniel Sorano): 1.0


Les attributs annuaire.nom_etablissement et bddtop.toponyme sont identiques

In [29]:
X=s_annuaire_ecole_1['type_etablissement'].values[0]
Y=s_bddtopo_ecole_1['nature'].values[0]

print(f"similarité ({X}|{Y}): {similarite_cosinus(X,Y)}")

similarité (Ecole|Enseignement primaire): 0.0


### **📊 II] Vérification des Données**

1️- Vérifier qu’on a bien toutes les infos :  D’abord, on a listé les infos importantes qu’on devait avoir (nom des écoles, localisation, potentiel solaire, etc.), puis on a vérifié que chaque base de données les contenait bien.
Quand certaines colonnes manquaient ou avaient un nom différent, on a ajusté pour que tout colle ensemble.

2- Se concentrer sur Saint-Denis : 

On ne voulait pas traiter toutes les données de France, alors on a filtré uniquement les écoles situées à Saint-Denis.

    On a pris les établissements scolaires depuis l’Annuaire officiel.
    On a extrait les écoles de la BDD Topo (qui regroupe des infos sur les bâtiments et les zones d’activité).
    On a isolé les données sur le potentiel solaire des bâtiments scolaires.

33- Vérifier la qualité des données : On a regardé si certaines infos étaient manquantes et combien d’écoles restaient après filtrage.
L’idée était de s’assurer qu’on pouvait faire une analyse fiable sans trop de trous dans les données.

4- Comparer les bases entre elles : 

Comme on avait plusieurs sources d’infos, on a cherché à voir si elles se recoupaient :

    Est-ce que les écoles de l’Annuaire sont bien dans la BDD Topo ?
    Est-ce qu’on retrouve les mêmes noms d’établissements ?
    Y a-t-il des écoles qui apparaissent dans une base mais pas dans une autre ?

5- Vérifier que tout est à jour : Pour finir, on a regardé les dates de mise à jour des données et affiché un petit aperçu des résultats 


In [30]:
# Vérification de la présence des attributs clés dans chaque dataset
checklist = {
    "annuaire": ["nom_etablissement", "type_etablissement", "code_commune", "date_maj_ligne"],
    "potentiel_gisement_solaire": ["c_cainsee", "surface", "irradiance"],
    "potentiel": ["surface", "productivite", "efficacite"],
    "communes": ["code_insee", "nom"],
    "zone_d_activite": ["categorie", "toponyme", "date_modification"]
}

# Vérification
missing_attributes = {}
for dataset_name, attributes in checklist.items():
    dataset = locals().get(dataset_name)  # Récupération du dataset par son nom
    if dataset is not None:
        missing_attrs = [attr for attr in attributes if attr not in dataset.columns]
        if missing_attrs:
            missing_attributes[dataset_name] = missing_attrs

# Affichage des résultats
if missing_attributes:
    print("\n Certains attributs sont manquants dans les datasets :")
    for dataset, attrs in missing_attributes.items():
        print(f"- {dataset} : {attrs}")
else:
    print("\nTous les attributs attendus sont présents dans chaque dataset.")



 Certains attributs sont manquants dans les datasets :
- potentiel_gisement_solaire : ['surface', 'irradiance']
- potentiel : ['surface', 'productivite', 'efficacite']
- communes : ['nom']


In [31]:
# Vérification des colonnes disponibles dans chaque dataset
datasets = {
    "potentiel_gisement_solaire": potentiel_gisement_solaire,
    "potentiel": potentiel,
    "communes": communes
}

for dataset_name, dataset in datasets.items():
    print(f"\n Colonnes disponibles dans {dataset_name}:")
    print(dataset.columns.tolist()) 


 Colonnes disponibles dans potentiel_gisement_solaire:
['geo_point_2d', 'geo_shape', 'objectid', 'n_sq_eb', 'n_sq_eb_or', 'c_cainsee', 'm2_e_in700', 'm2_e_70_80', 'm2_e_80_90', 'm2_e_90_10', 'm2_e_s1000', 'm2_e_tot', 'c_ens_moy', 'shape_leng', 'shape_area', 'geometry']

 Colonnes disponibles dans potentiel:
['objectid', 'id', 'nature', 'surf_util', 'indic2', 'gisement', 'eq_pano', 'eq_surf', 'systeme', 'protection', 'mos2017', 'insee', 'moyenne2', 'forme', 'production', 'mos17', 'st_area(shape)', 'st_length(shape)', 'geometry']

 Colonnes disponibles dans communes:
['cleabs', 'code_insee', 'code_insee_de_l_arrondissement', 'code_insee_de_la_collectivite_terr', 'code_insee_du_departement', 'code_insee_de_la_region', 'population', 'surface_en_ha', 'date_creation', 'date_modification', 'date_d_apparition', 'date_de_confirmation', 'code_postal', 'nom_officiel', 'chef_lieu_d_arrondissement', 'chef_lieu_de_collectivite_terr', 'chef_lieu_de_departement', 'chef_lieu_de_region', 'capitale_d_et

In [32]:
# Nouvelle checklist avec les colonnes adaptées
checklist_adaptee = {
    "annuaire": ["nom_etablissement", "type_etablissement", "code_commune", "date_maj_ligne"],
    "potentiel_gisement_solaire": ["c_cainsee", "m2_e_tot", "c_ens_moy"],  # Adaptation pour surface et irradiance
    "potentiel": ["surf_util", "production", "eq_surf"],  # Adaptation pour surface, productivité, efficacité
    "communes": ["code_insee", "nom_officiel"],  # Adaptation pour le nom officiel
    "zone_d_activite": ["categorie", "toponyme", "date_modification"]
}

# Vérification avec la nouvelle checklist
missing_attributes_adaptee = {}
for dataset_name, attributes in checklist_adaptee.items():
    dataset = locals().get(dataset_name)  # Récupération du dataset par son nom
    if dataset is not None:
        missing_attrs = [attr for attr in attributes if attr not in dataset.columns]
        if missing_attrs:
            missing_attributes_adaptee[dataset_name] = missing_attrs

# Affichage des résultats
if missing_attributes_adaptee:
    print("\nCertains attributs sont toujours manquants après adaptation :")
    for dataset, attrs in missing_attributes_adaptee.items():
        print(f"- {dataset} : {attrs}")
else:
    print("\nTous les attributs attendus sont maintenant présents dans chaque dataset.")



Tous les attributs attendus sont maintenant présents dans chaque dataset.


In [33]:
# On garde que les données pour Saint-Denis (93066)
saint_denis_annuaire = annuaire[annuaire.code_commune == "93066"]
saint_denis_gisement = potentiel_gisement_solaire[potentiel_gisement_solaire.c_cainsee == 93066]
saint_denis_potentiel = potentiel[potentiel.insee == "93066"]
saint_denis_communes = communes[communes.code_insee == "93066"]
saint_denis_zone = zone_d_activite[zone_d_activite.categorie == "Science et enseignement"]

# On affiche combien d’éléments on a après filtrage
print("\n Filtrage terminé, voici ce qu'on a trouvé :")
print(f"- Établissements (Annuaire) : {len(saint_denis_annuaire)}")
print(f"- Potentiel Gisement Solaire : {len(saint_denis_gisement)}")
print(f"- Potentiel : {len(saint_denis_potentiel)}")
print(f"- Communes : {len(saint_denis_communes)}")
print(f"- Zones d'activités (enseignement) : {len(saint_denis_zone)}")

# On affiche un petit aperçu des données filtrées
print("\n Un petit aperçu des données :")

print("\n Annuaire (premières lignes) :")
print(saint_denis_annuaire.head())

print("\n Potentiel Gisement Solaire :")
print(saint_denis_gisement.head())

print("\n Potentiel :")
print(saint_denis_potentiel.head())

print("\n Communes :")
print(saint_denis_communes.head())

print("\n Zones d'activités (enseignement) :")
print(saint_denis_zone.head())



 Filtrage terminé, voici ce qu'on a trouvé :
- Établissements (Annuaire) : 108
- Potentiel Gisement Solaire : 12319
- Potentiel : 0
- Communes : 1
- Zones d'activités (enseignement) : 4893

 Un petit aperçu des données :

 Annuaire (premières lignes) :
      identifiant_de_l_etablissement                nom_etablissement  \
10884                       0930168C  Ecole élémentaire Louise Michel   
10894                       0930208W   Ecole élémentaire Jules Guesde   
10910                       0930331E   Ecole élémentaire Jules Vallès   
10924                       0930393X  Ecole élémentaire Paul Langevin   
10938                       0930432P       Ecole maternelle Corbillon   

      type_etablissement statut_public_prive                  adresse_1  \
10884              Ecole              Public   35 rue Danielle Casanova   
10894              Ecole              Public         8 rue du Corbillon   
10910              Ecole              Public  55 boulevard Jules Guesde   
10924  

In [34]:
# Vérifier les valeurs uniques dans la colonne 'insee' du dataset 'Potentiel'
print("\n Valeurs uniques dans la colonne 'insee' du dataset Potentiel :")
print(potentiel["insee"].unique())



 Valeurs uniques dans la colonne 'insee' du dataset Potentiel :
[95078 95211 95370 95607 95658 95394 95218 95051 95572 95488 95592 95355
 95459 95280 95039 95625 95023 95529 95487 95341 95177 95379 95539 95446
 95134 95056 95308 95212 95652 95594 95026 95018 95042 95019 95598 95582
 95660 95028 95500 95535 95510 95002 95101 95523 95257 95176 95491 95306
 95127 95277 95476 95628 95120 95203 95610 95213 95119 95298 95480 95166
 95303 95627 95328 95424 95680 95268 95430 95527 95585 95091 95409 95088
 95252 95063 95149 95219 95256 95169 95580 95351 95052 95675 95566 95116
 95504 95058 95313 95452 95445 95678 95197 95288 95574 95555 95428 95369
 95395 95205 95094 95492 95483 95102 95270 95450 95170 95348 95541 95040
 95429 95011 95554 95012 95229 95426 95199 95060 95612 95271 95287 95142
 95331 95210 95637 95353]


In [35]:
# Afficher les noms des colonnes du dataset Potentiel
print("\n Liste des colonnes disponibles dans Potentiel :")
print(potentiel.columns)

# Afficher les 5 premières lignes pour voir s'il y a un indice
print("\n Aperçu du dataset Potentiel :")
print(potentiel.head())



 Liste des colonnes disponibles dans Potentiel :
Index(['objectid', 'id', 'nature', 'surf_util', 'indic2', 'gisement',
       'eq_pano', 'eq_surf', 'systeme', 'protection', 'mos2017', 'insee',
       'moyenne2', 'forme', 'production', 'mos17', 'st_area(shape)',
       'st_length(shape)', 'geometry'],
      dtype='object')

 Aperçu du dataset Potentiel :
   objectid                        id               nature  surf_util  indic2  \
0         1  BATIMENT0000000356480536  Bâtiment industriel     382.50       3   
1         2  BATIMENT0000000356480545  Bâtiment industriel      83.25       2   
2         3  BATIMENT0000000356475551  Bâtiment industriel     213.75       3   
3         4  BATIMENT0000000317228796  Bâtiment industriel     240.75       3   
4         5  BATIMENT0000000003786796  Bâtiment industriel     132.75       3   

        gisement              eq_pano             eq_surf  \
0      important  plus de 50 panneaux      plus de 115 m2   
1  intermédiaire     10 à 50 panne

In [36]:
# Vérification de la couverture temporelle des données
print("\nVérification des dates de mise à jour dans les datasets :")

# Annuaire - Écoles
if "date_maj_ligne" in saint_denis_annuaire.columns:
    print("\nAnnuaire (écoles) - Dates de mise à jour uniques :")
    print(saint_denis_annuaire["date_maj_ligne"].unique())

# Zones d'activités - Science et enseignement
if "date_modification" in saint_denis_zone.columns:
    print("\nZones d'activités (enseignement) - Dates de modification uniques :")
    print(saint_denis_zone["date_modification"].dropna().sort_values().unique())

# Communes
if "date_modification" in saint_denis_communes.columns:
    print("\nCommunes - Dates de modification uniques :")
    print(saint_denis_communes["date_modification"].dropna().sort_values().unique())



Vérification des dates de mise à jour dans les datasets :

Annuaire (écoles) - Dates de mise à jour uniques :
<DatetimeArray>
['2025-02-03 00:00:00']
Length: 1, dtype: datetime64[ms]

Zones d'activités (enseignement) - Dates de modification uniques :
<DatetimeArray>
['2021-08-03 13:11:01.290000', '2022-10-04 08:24:44.542000',
 '2022-12-05 12:26:55.418000', '2022-12-06 13:38:05.496000',
 '2023-01-11 23:10:34.898000', '2023-01-13 23:13:57.836000',
 '2023-01-17 22:31:21.398000', '2023-01-24 18:16:12.620000',
 '2023-01-26 13:49:36.128000', '2023-02-16 17:43:52.478000',
 ...
 '2024-11-27 10:24:01.306000', '2024-11-27 18:25:27.752000',
 '2024-11-27 18:25:43.732000', '2024-11-27 18:48:18.344000',
 '2024-11-29 15:03:31.465000', '2024-12-02 13:58:13.202000',
 '2024-12-02 16:40:09.789000', '2024-12-04 11:48:56.481000',
 '2024-12-04 12:46:39.170000', '2024-12-05 08:50:36.735000']
Length: 929, dtype: datetime64[ms]

Communes - Dates de modification uniques :
<DatetimeArray>
['2024-05-24 14:34:47.

In [37]:
# Comparaison des noms d'établissements entre Annuaire et BDD Topo

# On garde uniquement les écoles dans l'annuaire
annuaire_ecoles = saint_denis_annuaire[saint_denis_annuaire["type_etablissement"] == "Ecole"]

# On applique une normalisation sur les noms d'établissements
annuaire_ecoles["nom_normalise"] = annuaire_ecoles["nom_etablissement"].apply(lambda x: unidecode(str(x)).lower())
zone_d_activite["toponyme_normalise"] = zone_d_activite["toponyme"].apply(lambda x: unidecode(str(x)).lower())

# On cherche les correspondances entre les deux datasets
comparaison = annuaire_ecoles.merge(
    zone_d_activite,
    left_on="nom_normalise",
    right_on="toponyme_normalise",
    how="left"
)

# Affichage des différences
print("\nComparaison entre Annuaire et BDD Topo :")
print(comparaison[["nom_etablissement", "type_etablissement", "toponyme", "categorie"]].dropna().head(10))

# Établissements de l'annuaire qui ne sont pas dans BDD Topo
manquants = comparaison[comparaison["toponyme"].isna()]
print("\nÉtablissements présents dans l'Annuaire mais absents de BDD Topo :")
print(manquants[["nom_etablissement", "type_etablissement"]].head(10))

# Établissements de BDD Topo qui ne sont pas dans l'Annuaire
topo_non_trouves = zone_d_activite[~zone_d_activite["toponyme_normalise"].isin(annuaire_ecoles["nom_normalise"])]
print("\nÉtablissements présents dans BDD Topo mais absents de l'Annuaire :")
print(topo_non_trouves[["toponyme", "categorie"]].head(10))



Comparaison entre Annuaire et BDD Topo :
                 nom_etablissement type_etablissement  \
0  Ecole élémentaire Louise Michel              Ecole   
1  Ecole élémentaire Louise Michel              Ecole   
2   Ecole élémentaire Jules Guesde              Ecole   
3   Ecole élémentaire Jules Guesde              Ecole   
4   Ecole élémentaire Jules Guesde              Ecole   
5   Ecole élémentaire Jules Vallès              Ecole   
6   Ecole élémentaire Jules Vallès              Ecole   
7   Ecole élémentaire Jules Vallès              Ecole   
8   Ecole élémentaire Jules Vallès              Ecole   
9   Ecole élémentaire Jules Vallès              Ecole   

                          toponyme                categorie  
0  École Élémentaire Louise Michel  Science et enseignement  
1  École Élémentaire Louise Michel  Science et enseignement  
2   École Élémentaire Jules Guesde  Science et enseignement  
3   École Élémentaire Jules Guesde  Science et enseignement  
4   École Élémentair

c:\Users\eteiw\Desktop\amel\algorithme\.venv\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [43]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Filtrer les écoles dans la base de données topo
zone_d_activite_ecoles = zone_d_activite[zone_d_activite["categorie"] == "Science et enseignement"].copy()
print(f"Nombre d'écoles après filtrage : {len(zone_d_activite_ecoles)}")

# Filtrer les écoles dans l'annuaire
annuaire_ecoles = saint_denis_annuaire[saint_denis_annuaire["type_etablissement"] == "Ecole"].copy()

# Nettoyer les noms (sans accents, en minuscules)
annuaire_ecoles["nom_normalise"] = annuaire_ecoles["nom_etablissement"].apply(lambda x: unidecode(str(x)).lower())
zone_d_activite_ecoles["toponyme_normalise"] = zone_d_activite_ecoles["toponyme"].apply(lambda x: unidecode(str(x)).lower())

# Comparer les noms avec un modèle de texte
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(
    annuaire_ecoles["nom_normalise"].tolist() + zone_d_activite_ecoles["toponyme_normalise"].tolist()
)

# Séparer les résultats
annuaire_vectors = tfidf_matrix[:len(annuaire_ecoles)]
zone_vectors = tfidf_matrix[len(annuaire_ecoles):]

# Comparer les noms des écoles des deux bases
similarities = cosine_similarity(annuaire_vectors, zone_vectors)

# Trouver les meilleures correspondances
matches = []
for i, nom_ecole in enumerate(annuaire_ecoles["nom_etablissement"]):
    best_match_idx = similarities[i].argmax()
    best_match_score = similarities[i][best_match_idx]
    if best_match_score > 0.7:  # Si la correspondance est forte
        matches.append({
            "Ecole Annuaire": nom_ecole,
            "Ecole BDD Topo": zone_d_activite_ecoles.iloc[best_match_idx]["toponyme"],
            "Score": round(best_match_score, 2)
        })

# Afficher les résultats
matches_df = pd.DataFrame(matches).sort_values(by="Score", ascending=False)
print("\nÉcoles correspondantes entre les deux bases :")
print(matches_df.head(10))

# Trouver les écoles qui n'ont pas de correspondance
ecoles_sans_match = annuaire_ecoles[~annuaire_ecoles["nom_etablissement"].isin(matches_df["Ecole Annuaire"])]

print("\nÉcoles sans correspondance :")
print(ecoles_sans_match[["nom_etablissement"]].head(10))


Nombre d'écoles après filtrage : 4893

Écoles correspondantes entre les deux bases :
                          Ecole Annuaire  \
0        Ecole élémentaire Louise Michel   
1         Ecole élémentaire Jules Guesde   
2         Ecole élémentaire Jules Vallès   
3        Ecole élémentaire Paul Langevin   
4             Ecole maternelle Corbillon   
5              Ecole maternelle Marville   
8      Ecole élémentaire Albert Calmette   
10  Ecole maternelle Delaunay-Belleville   
9           Ecole maternelle L'Hermitage   
16           Ecole maternelle Le Corduan   

                          Ecole BDD Topo  Score  
0        École Élémentaire Louise Michel    1.0  
1         École Élémentaire Jules Guesde    1.0  
2         École Élémentaire Jules Vallès    1.0  
3      École Élémentaire Paul Langevin 2    1.0  
4             École Maternelle Corbillon    1.0  
5              École Maternelle Marville    1.0  
8      École Élémentaire Albert Calmette    1.0  
10  École Maternelle Delaunay-

In [ ]:
# Afficher les colonnes de BDD Topo pour vérifier si "code_insee" existe
print("Colonnes disponibles dans BDD Topo :")
print(zone_d_activite.columns)


Colonnes disponibles dans BDD Topo :
Index(['cleabs', 'categorie', 'nature', 'nature_detaillee', 'toponyme',
       'statut_du_toponyme', 'importance', 'fictif', 'etat_de_l_objet',
       'date_creation', 'date_modification', 'date_d_apparition',
       'date_de_confirmation', 'sources', 'identifiants_sources',
       'methode_d_acquisition_planimetrique', 'precision_planimetrique',
       'identifiant_voie_ban', 'nom_commercial', 'geometry',
       'toponyme_normalise'],
      dtype='object')


In [ ]:
# Rechercher les établissements contenant "Saint-Denis" dans le toponyme
print("\nÉtablissements contenant 'Saint-Denis' dans la colonne 'toponyme' :")
print(zone_d_activite[zone_d_activite["toponyme"].str.contains("Saint-Denis", case=False, na=False)].head(10))

# Rechercher les établissements contenant "Saint-Denis" dans le nom commercial (s'il est rempli)
if "nom_commercial" in zone_d_activite.columns:
    print("\nÉtablissements contenant 'Saint-Denis' dans la colonne 'nom_commercial' :")
    print(zone_d_activite[zone_d_activite["nom_commercial"].str.contains("Saint-Denis", case=False, na=False)].head(10))



Établissements contenant 'Saint-Denis' dans la colonne 'toponyme' :
                        cleabs                   categorie  \
75    SURFACTI0000000002556676  Administratif ou militaire   
347   SURFACTI0000000002556752  Administratif ou militaire   
363   SURFACTI0000002215709478  Administratif ou militaire   
875   SURFACTI0000000334649991  Administratif ou militaire   
1297  SURFACTI0000000002556733  Administratif ou militaire   
1328  SURFACTI0000002008337372  Administratif ou militaire   
1401  SURFACTI0000000001728128                   Religieux   
1465  SURFACTI0000000000140738                   Religieux   
2747  SURFACTI0000000002556828  Administratif ou militaire   
2851  SURFACTI0000000000142274          Culture et loisirs   

                                   nature      nature_detaillee  \
75            Etablissement pénitentiaire        Maison d'arrêt   
347                                 Poste       Bureau de poste   
363                                Police  Comm

In [ ]:
# Afficher les colonnes disponibles dans zone_d_activite pour identifier la bonne colonne
print(zone_d_activite.columns)


Index(['cleabs', 'categorie', 'nature', 'nature_detaillee', 'toponyme',
       'statut_du_toponyme', 'importance', 'fictif', 'etat_de_l_objet',
       'date_creation', 'date_modification', 'date_d_apparition',
       'date_de_confirmation', 'sources', 'identifiants_sources',
       'methode_d_acquisition_planimetrique', 'precision_planimetrique',
       'identifiant_voie_ban', 'nom_commercial', 'geometry',
       'toponyme_normalise'],
      dtype='object')


In [45]:

# Vérifier si la colonne "toponyme" existe avant de filtrer
if "toponyme" in zone_d_activite.columns:
    # Garder seulement les écoles de "Science et enseignement" situées à Saint-Denis
    ecoles_bddtopo = zone_d_activite[
        (zone_d_activite["categorie"] == "Science et enseignement") & 
        (zone_d_activite["toponyme"].str.contains("Saint-Denis", case=False, na=False))
    ].copy()
else:
    print("Erreur : colonne 'toponyme' introuvable dans zone_d_activite.")
    ecoles_bddtopo = pd.DataFrame()  # Pour éviter les erreurs si BDD Topo est vide

# Garder seulement les écoles de Saint-Denis dans l'annuaire
ecoles_annuaire = saint_denis_annuaire[
    (saint_denis_annuaire["type_etablissement"] == "Ecole") & 
    (saint_denis_annuaire["code_commune"] == "93066")
].copy()

# Transformer les noms en minuscules et sans accents pour mieux comparer
ecoles_annuaire["nom_normalise"] = ecoles_annuaire["nom_etablissement"].apply(lambda x: unidecode(str(x)).lower())
if not ecoles_bddtopo.empty:
    ecoles_bddtopo["toponyme_normalise"] = ecoles_bddtopo["toponyme"].apply(lambda x: unidecode(str(x)).lower())

# Comparer les écoles des deux bases
for nom_ecole in ecoles_annuaire["nom_normalise"].unique():
    # Sélection des écoles correspondantes dans l'annuaire
    ecoles_trouvees_annuaire = ecoles_annuaire[ecoles_annuaire["nom_normalise"].str.contains(nom_ecole, case=False, na=False)]
    
    # Vérifier si on trouve une correspondance dans la BDD Topo
    if not ecoles_bddtopo.empty:
        ecoles_trouvees_bddtopo = ecoles_bddtopo[ecoles_bddtopo["toponyme_normalise"].str.contains(nom_ecole, case=False, na=False)]
    else:
        ecoles_trouvees_bddtopo = pd.DataFrame()  # Si BDD Topo est vide, éviter l'erreur

    # Afficher les résultats
    if not ecoles_trouvees_bddtopo.empty:
        print(f"\n✔ École trouvée dans les deux bases : {nom_ecole}")
        print("Annuaire :")
        print(ecoles_trouvees_annuaire[["nom_etablissement", "type_etablissement"]])
        print("BDD Topo :")
        print(ecoles_trouvees_bddtopo[["toponyme", "nature"]])
    else:
        print(f"\n École absente de la BDD Topo : {nom_ecole}")





 École absente de la BDD Topo : ecole elementaire louise michel

 École absente de la BDD Topo : ecole elementaire jules guesde

 École absente de la BDD Topo : ecole elementaire jules valles

 École absente de la BDD Topo : ecole elementaire paul langevin

 École absente de la BDD Topo : ecole maternelle corbillon

 École absente de la BDD Topo : ecole maternelle marville

 École absente de la BDD Topo : ecole maternelle la montjoie

 École absente de la BDD Topo : ecole maternelle le lendit

 École absente de la BDD Topo : ecole elementaire albert calmette

 École absente de la BDD Topo : ecole maternelle l'hermitage

 École absente de la BDD Topo : ecole maternelle delaunay-belleville

 École absente de la BDD Topo : ecole maternelle les drapiers

 École absente de la BDD Topo : ecole elementaire maria casares

 École absente de la BDD Topo : ecole primaire opaline - suzanne lacore

 École absente de la BDD Topo : ecole primaire la roseraie - jacqueline de chambrun

 École absente 

 Lister les attributs importants pour évaluer le potentiel solaire des écoles

In [46]:
# Affichage des attributs importants pour le potentiel solaire des écoles à Saint-Denis
print("Attributs Importants pour évaluer le potentiel solaire des écoles à Saint-Denis :")
print("""
Annuaire des écoles :
- nom_etablissement
- type_etablissement
- code_commune
- geometry
- date_maj_ligne

BDD Topo :
- toponyme
- categorie
- geometry
- date_modification

Potentiel Solaire :
- surf_util (surface)
- production (productivite)
- moyenne2 (irradiance)
- geometry
""")

# Sélectionner les écoles de Saint-Denis dans l'Annuaire
annuaire_saint_denis = annuaire[annuaire["code_commune"] == "93066"]

# Sélectionner les écoles de Saint-Denis dans BDD Topo
bddtopo_saint_denis = zone_d_activite[
    (zone_d_activite["categorie"] == "Science et enseignement") & 
    (zone_d_activite["toponyme"].str.contains("Saint-Denis", case=False, na=False))
].copy()

# Sélectionner les écoles de Saint-Denis dans le dataset du Potentiel Solaire
potentiel_saint_denis = potentiel[potentiel["geometry"].within(annuaire_saint_denis.geometry.unary_union)]

# Afficher un aperçu des données filtrées

# Aperçu des écoles de Saint-Denis dans l'Annuaire
print("\nAperçu des écoles de Saint-Denis dans l'Annuaire des écoles :")
print(annuaire_saint_denis[["nom_etablissement", "type_etablissement", "geometry"]].head())

# Aperçu des écoles de Saint-Denis dans BDD Topo
print("\nAperçu des écoles de Saint-Denis dans BDD Topo :")
print(bddtopo_saint_denis[["toponyme", "categorie", "geometry"]].head())

# Aperçu du Potentiel Solaire pour Saint-Denis
# Correction : Nous utilisons les colonnes appropriées du dataset 'potentiel'
print("\nAperçu du Potentiel Solaire pour Saint-Denis :")
print(potentiel_saint_denis[["surf_util", "production", "moyenne2", "geometry"]].head())



Attributs Importants pour évaluer le potentiel solaire des écoles à Saint-Denis :

Annuaire des écoles :
- nom_etablissement
- type_etablissement
- code_commune
- geometry
- date_maj_ligne

BDD Topo :
- toponyme
- categorie
- geometry
- date_modification

Potentiel Solaire :
- surf_util (surface)
- production (productivite)
- moyenne2 (irradiance)
- geometry


Aperçu des écoles de Saint-Denis dans l'Annuaire des écoles :
                     nom_etablissement type_etablissement  \
10884  Ecole élémentaire Louise Michel              Ecole   
10894   Ecole élémentaire Jules Guesde              Ecole   
10910   Ecole élémentaire Jules Vallès              Ecole   
10924  Ecole élémentaire Paul Langevin              Ecole   
10938       Ecole maternelle Corbillon              Ecole   

                       geometry  
10884  POINT (2.36427 48.92859)  
10894  POINT (2.35373 48.93804)  
10910  POINT (2.35023 48.93775)  
10924  POINT (2.36214 48.94165)  
10938  POINT (2.35349 48.93802)  

Ape

C:\Users\eteiw\AppData\Local\Temp\ipykernel_112704\2954488311.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  potentiel_saint_denis = potentiel[potentiel["geometry"].within(annuaire_saint_denis.geometry.unary_union)]


In [ ]:
# Vérifier les colonnes de Potentiel Solaire
print(potentiel_saint_denis.columns)


Index(['objectid', 'id', 'nature', 'surf_util', 'indic2', 'gisement',
       'eq_pano', 'eq_surf', 'systeme', 'protection', 'mos2017', 'insee',
       'moyenne2', 'forme', 'production', 'mos17', 'st_area(shape)',
       'st_length(shape)', 'geometry'],
      dtype='object')


In [ ]:
# Colonnes utiles pour chaque base de données 
important_columns = {
    "Annuaire des écoles": ["nom_etablissement", "type_etablissement", "code_commune", "geometry", "date_maj_ligne"],
    "BDD Topo": [
        "insee", "moyenne2", "surf_util", "st_area(shape)", "st_length(shape)", "geometry"
    ],
    "Potentiel Solaire": ["surf_util", "production", "geometry"]
}

# Fonction pour voir combien de valeurs sont manquantes dans chaque colonne (sauf geometry)
def calculate_missing_rate(df, columns):
    # Exclure la colonne 'geometry' pour éviter l'erreur avec les objets géométriques
    columns_to_check = [col for col in columns if col != 'geometry']
    return df[columns_to_check].isnull().mean() * 100

# Filtrer les données pour Saint-Denis (code INSEE 93066)
# Annuaire des écoles : filtrer par code_commune
annuaire_saint_denis = annuaire[annuaire["code_commune"] == "93066"]

# Vérification si la colonne "insee" est présente dans BDD Topo
if "insee" in zone_d_activite.columns:
    # Si "insee" existe, filtrer par code INSEE dans la colonne 'insee'
    bddtopo_saint_denis = zone_d_activite[
        zone_d_activite["insee"] == "93066"
    ]
else:
    # Sinon, afficher un message d'erreur
    print("Colonne 'insee' introuvable dans BDD Topo, vérifiez le nom exact de la colonne.")

# Potentiel Solaire : on filtre en utilisant la géométrie des écoles de Saint-Denis
potentiel_saint_denis = potentiel[potentiel["geometry"].within(annuaire_saint_denis.geometry.unary_union)]

# Calculer les taux de valeurs manquantes pour chaque dataset

print("Taux de valeurs manquantes :")

# Annuaire des écoles
print("\nAnnuaire des écoles à Saint-Denis :")
print(calculate_missing_rate(annuaire_saint_denis, important_columns["Annuaire des écoles"]))

# BDD Topo
if "insee" in zone_d_activite.columns:
    print("\nBDD Topo à Saint-Denis :")
    print(calculate_missing_rate(bddtopo_saint_denis, important_columns["BDD Topo"]))

# Potentiel Solaire
print("\nPotentiel Solaire à Saint-Denis :")
print(calculate_missing_rate(potentiel_saint_denis, important_columns["Potentiel Solaire"]))

# Complément : afficher les aperçus de ces données filtrées pour s'assurer que les données sont correctes

print("\nAperçu des écoles de Saint-Denis dans l'Annuaire des écoles :")
print(annuaire_saint_denis[["nom_etablissement", "type_etablissement", "geometry"]].head())

if "insee" in zone_d_activite.columns:
    print("\nAperçu des écoles de Saint-Denis dans BDD Topo :")
    print(bddtopo_saint_denis[["insee", "moyenne2", "surf_util", "st_area(shape)", "st_length(shape)", "geometry"]].head())

print("\nAperçu du Potentiel Solaire pour Saint-Denis :")
print(potentiel_saint_denis[["surf_util", "production", "geometry"]].head())



Colonne 'insee' introuvable dans BDD Topo, vérifiez le nom exact de la colonne.
Taux de valeurs manquantes :

Annuaire des écoles à Saint-Denis :
nom_etablissement     0.0
type_etablissement    0.0
code_commune          0.0
date_maj_ligne        0.0
dtype: float64

Potentiel Solaire à Saint-Denis :
surf_util    NaN
production   NaN
dtype: float64

Aperçu des écoles de Saint-Denis dans l'Annuaire des écoles :
                     nom_etablissement type_etablissement  \
10884  Ecole élémentaire Louise Michel              Ecole   
10894   Ecole élémentaire Jules Guesde              Ecole   
10910   Ecole élémentaire Jules Vallès              Ecole   
10924  Ecole élémentaire Paul Langevin              Ecole   
10938       Ecole maternelle Corbillon              Ecole   

                       geometry  
10884  POINT (2.36427 48.92859)  
10894  POINT (2.35373 48.93804)  
10910  POINT (2.35023 48.93775)  
10924  POINT (2.36214 48.94165)  
10938  POINT (2.35349 48.93802)  

Aperçu du Potent

C:\Users\eteiw\AppData\Local\Temp\ipykernel_95760\2369940402.py:31: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  potentiel_saint_denis = potentiel[potentiel["geometry"].within(annuaire_saint_denis.geometry.unary_union)]


In [ ]:
# Fonction pour calculer le taux de valeurs manquantes
def calculate_missing_rate(df, columns):
    # Exclure la colonne 'geometry' si elle existe
    columns_to_check = [col for col in columns if col != 'geometry']
    return df[columns_to_check].isnull().mean() * 100  # Retourne le pourcentage de valeurs manquantes

# Colonnes importantes pour chaque dataset
important_columns = {
    "Annuaire des écoles": ["nom_etablissement", "type_etablissement", "code_commune", "geometry", "date_maj_ligne"],
    "BDD Topo": ["toponyme", "categorie", "geometry", "date_modification"],
    "Potentiel Solaire": ["surf_util", "production", "moyenne2", "geometry"]
}

# Filtrage des datasets pour Saint-Denis
annuaire_saint_denis = annuaire[annuaire["code_commune"] == "93066"]
bddtopo_saint_denis = zone_d_activite[zone_d_activite["toponyme"].str.contains("Saint-Denis", case=False, na=False)]
potentiel_saint_denis = potentiel[potentiel["geometry"].within(annuaire_saint_denis.geometry.unary_union)]

# Calcul des taux de valeurs manquantes
print("Taux de valeurs manquantes :")

# Annuaire des écoles
print("\nAnnuaire des écoles à Saint-Denis :")
print(calculate_missing_rate(annuaire_saint_denis, important_columns["Annuaire des écoles"]))

# BDD Topo
print("\nBDD Topo à Saint-Denis :")
print(calculate_missing_rate(bddtopo_saint_denis, important_columns["BDD Topo"]))

# Potentiel Solaire
print("\nPotentiel Solaire à Saint-Denis :")
print(calculate_missing_rate(potentiel_saint_denis, important_columns["Potentiel Solaire"]))

Taux de valeurs manquantes :

Annuaire des écoles à Saint-Denis :
nom_etablissement     0.0
type_etablissement    0.0
code_commune          0.0
date_maj_ligne        0.0
dtype: float64

BDD Topo à Saint-Denis :
toponyme             0.0
categorie            0.0
date_modification    0.0
dtype: float64

Potentiel Solaire à Saint-Denis :
surf_util    NaN
production   NaN
moyenne2     NaN
dtype: float64


C:\Users\eteiw\AppData\Local\Temp\ipykernel_95760\686619837.py:17: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  potentiel_saint_denis = potentiel[potentiel["geometry"].within(annuaire_saint_denis.geometry.unary_union)]
